I will start with small 2 qubit problems, one solvable and one not.

# The set

In [1]:
solvable_problem = [1,1] #An solvable instance of the partitioning problem to be solved using 2 quibts
unsolvable_problem = [1,3] #An unsolvable instance of the partitioning problem to be solved using 2 quibts

# Qiskit

In [2]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 6.0 MB/s 
     |████████████████████████████████| 19.2 MB 7.7 MB/s 
     |████████████████████████████████| 240 kB 43.0 MB/s 
     |████████████████████████████████| 106 kB 37.8 MB/s 
     |████████████████████████████████| 55 kB 1.6 MB/s 
     |████████████████████████████████| 49 kB 4.1 MB/s 
     |████████████████████████████████| 37.5 MB 1.3 MB/s 
     |████████████████████████████████| 943 kB 45.6 MB/s 
     |████████████████████████████████| 50 kB 3.1 MB/s 
     |████████████████████████████████| 4.1 MB 46.7 MB/s 
     |████████████████████████████████| 1.9 MB 44.0 MB/s 
     |████████████████████████████████| 112 kB 54.0 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.2-py3-none-any.whl size=12224 sha256=3611415640c3e63b4327ba561125fa4c6975afa1e971436bd054a5e092ad53ce
  Stored in directory: /root/.cache/pip/wheels/30/dc/90/0ee55

In [3]:
from qiskit.algorithms import VQE
from qiskit import Aer, IBMQ
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.providers.aer.noise import NoiseModel
from qiskit.algorithms.optimizers import L_BFGS_B, COBYLA
from qiskit.tools.visualization import circuit_drawer
from qiskit.circuit.library import EfficientSU2  

seed = 9999
entanglement = "linear"

## preparing the simulator

In [4]:
def prepare_simulator(simulator_name, optimizer_name, seed):
    algorithm_globals.random_seed = seed
    backend = Aer.get_backend(simulator_name)
    quantum_instance = QuantumInstance(backend=backend, 
                                      shots=8192, 
                                      seed_transpiler=seed, 
                                      seed_simulator=seed
                                      )
    if optimizer_name == "cobyla" or optimizer_name == "COBYLA":
      optimizer = COBYLA(maxiter=60)
    else:
      optimizer = L_BFGS_B(maxiter=60)
    return optimizer, quantum_instance

## preparing the hamiltonian for 2 quibts (Ising model)

In [5]:
import numpy as np
from qiskit.opflow import Z, I

def create_hamiltonian(S):
  n_1 = S[0]
  n_2 = S[1]

  H = n_1*n_1*(I^I) + n_1*n_2*(Z^Z) + n_2*n_1*(Z^Z) + n_2*n_2*(I^I)
  print("========== Ising Model Hamiltonian for Two Qubits ==========\n")
  print(H.to_matrix())
  print()
  return H

## compute minimal eigenvalue

In [6]:
from qiskit.circuit.library import EfficientSU2

def minimum_eigenvalue_by_vqe(num_qubits, ansatz_entanglement, optimizer, quantum_instance, hamiltonian):
    ansatz = EfficientSU2(num_qubits=num_qubits, entanglement=entanglement)
    #ansatz.draw(output='latex', scale=0.5)
    vqe = VQE(ansatz, optimizer, quantum_instance=quantum_instance)
    result = vqe.compute_minimum_eigenvalue(hamiltonian)
    print("minimal eigenvalue: ", format(result.eigenvalue,".3f"))

## run test

In [11]:
def run_experiment(simulator,optimizer,H,num_qubits=2):
  bfgs_optimizer, quantum_instance = prepare_simulator(simulator, optimizer, seed) 
  minimum_eigenvalue_by_vqe(num_qubits, entanglement, bfgs_optimizer, quantum_instance, H)
  return minimum_eigenvalue_by_vqe


In [17]:
def test_problem_instance(H, num_qubits=2):
  print(f"simulator: qasm, optimizer: cobyla, num of qubits: {num_qubits}", end = ' ') 
  run_experiment("qasm_simulator","cobyla",H,num_qubits)  
  print(f"simulator: qasm, optimizer: bfgs, num of qubits: {num_qubits}", end = ' ')                    
  run_experiment("qasm_simulator","bfgs",H, num_qubits)   
  print(f"simulator: aer_simulator_statevector, optimizer: cobyla, num of qubits: {num_qubits}", end = ' ') 
  run_experiment("aer_simulator_statevector","cobyla",H,num_qubits)  
  print(f"simulator: aer_simulator_statevector, optimizer: bfgs, num of qubits: {num_qubits}", end = ' ')                    
  run_experiment("aer_simulator_statevector","bfgs",H, num_qubits)  
       

# Run experiments

## Solvable problem ([1,1])

In [9]:
H = create_hamiltonian(solvable_problem)

========== Ising Model Hamiltonian for Two Qubits ==========

[[4.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 4.+0.j]]



In [18]:
test_problem_instance(H, num_qubits=2)

simulator: qasm, optimizer: cobyla, num of qubits: 2 minimal eigenvalue:  0.000+0.000j
simulator: qasm, optimizer: bfgs, num of qubits: 2 minimal eigenvalue:  1.247+0.000j
simulator: aer_simulator_statevector, optimizer: cobyla, num of qubits: 2 minimal eigenvalue:  0.002+0.000j
simulator: aer_simulator_statevector, optimizer: bfgs, num of qubits: 2 minimal eigenvalue:  0.000+0.000j


## Unsolvable problem ([1,3])

## compute minimal eigenvalue for unsolvable problem

In [19]:
H = create_hamiltonian(unsolvable_problem)

========== Ising Model Hamiltonian for Two Qubits ==========

[[16.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  4.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  4.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j 16.+0.j]]



In [20]:
test_problem_instance(H, num_qubits=2)

simulator: qasm, optimizer: cobyla, num of qubits: 2 minimal eigenvalue:  4.000+0.000j
simulator: qasm, optimizer: bfgs, num of qubits: 2 minimal eigenvalue:  7.740+0.000j
simulator: aer_simulator_statevector, optimizer: cobyla, num of qubits: 2 minimal eigenvalue:  4.004+0.000j
simulator: aer_simulator_statevector, optimizer: bfgs, num of qubits: 2 minimal eigenvalue:  4.000+0.000j


# Results for 2 qubits problems instances

All the experiments went well except of the combination of the qasm simulator and the bfgs optimizer, which returned result that deviated a lot from the expected result.

## Solvable problem ([1,1])








Optimizer/Simulator  | qasm               | aer_simulator_statevector
------------------|:----------------|--------------------
Cobyla            |  0.0 (worked!)  |  0.000 (worked!)
bfgs              |  1.247 (very wrong result)  |  0.000 (worked!)

## Unsolvable problem ([1,3])









Optimizer/Simulator  | qasm               | aer_simulator_statevector
------------------|:----------------|--------------------
Cobyla            |  4.0 (worked!)  |  4.000 (worked!)
bfgs              |  7.74 (very wrong result)  |  4.000 (worked!)

## General Form

In [21]:
solvable_problem_4_qubits = [1,2,3,2]
unsolvable_problem_4_qubits = [1,2,4,2]

### preparing the hamiltonian for n quibts (Ising model)

In [22]:
import numpy as np
from qiskit.opflow import Z, I

def create_general_hamiltonian(S):
  num_qubits = len(S)
  sum = 0
  sigmas_list = []
  for qubit in range(num_qubits):
    if qubit == 0:
       sigma = Z
    else:
       sigma = I
    for i in range(1,num_qubits):
      if i == qubit:
        sigma = sigma^Z
      else:
        sigma = sigma^I
    sigmas_list.append(sigma)  

  for i in range(len(sigmas_list)):
    for j in range(len(sigmas_list)):
      sum += S[i]*S[j]* sigmas_list[i]@sigmas_list[j]
  print("========== Ising Model Hamiltonian ==========\n")
  print(sum.to_matrix())
  print()
  return sum

# Run experiments

## Solvable problem ([1,2,3,2])

In [26]:
H = create_general_hamiltonian(solvable_problem_4_qubits)

========== Ising Model Hamiltonian ==========

[[64.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j 16.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  4.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  4.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j 16.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  4.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+

In [27]:
test_problem_instance(H, num_qubits=4)

simulator: qasm, optimizer: cobyla, num of qubits: 4 minimal eigenvalue:  3.665+0.000j
simulator: qasm, optimizer: bfgs, num of qubits: 4 minimal eigenvalue:  17.483+0.000j
simulator: aer_simulator_statevector, optimizer: cobyla, num of qubits: 4 minimal eigenvalue:  5.514+0.000j
simulator: aer_simulator_statevector, optimizer: bfgs, num of qubits: 4 minimal eigenvalue:  0.000+0.000j


## Unsolvable problem ([1,2,4,2])

## compute minimal eigenvalue for unsolvable problem

In [31]:
H = create_general_hamiltonian(unsolvable_problem_4_qubits)

========== Ising Model Hamiltonian ==========

[[81.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j 25.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  9.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j 25.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  9.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+

In [32]:
test_problem_instance(H, num_qubits=4)

simulator: qasm, optimizer: cobyla, num of qubits: 4 minimal eigenvalue:  6.538+0.000j
simulator: qasm, optimizer: bfgs, num of qubits: 4 minimal eigenvalue:  25.465+0.000j
simulator: aer_simulator_statevector, optimizer: cobyla, num of qubits: 4 minimal eigenvalue:  8.969+0.000j
simulator: aer_simulator_statevector, optimizer: bfgs, num of qubits: 4 minimal eigenvalue:  1.000+0.000j


# Results for 4 qubits problems instances

This time the only combination that worked without deviation is aer_simulator_statevector + bfgs optimizer

## Solvable problem ([1,2,3,2])








Optimizer/Simulator  | qasm               | aer_simulator_statevector
------------------|:----------------|--------------------
Cobyla            |  3.665 (very wrong result)  |  5.514 (very wrong result)
bfgs              |  17.483 (very wrong result)  |  0.000 (worked!)

## Unsolvable problem ([1,2,4,2])









Optimizer/Simulator  | qasm               | aer_simulator_statevector
------------------|:----------------|--------------------
Cobyla            |  6.538 (very wrong result)  |  8.969 (very wrong result)
bfgs              |  25.465 (very wrong result)  |  1.000 (worked!)